In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [13]:
import pandas as pd

csv_path = "/content/drive/MyDrive/NLP Project/Data/gsm8k_test.csv"
df = pd.read_csv(csv_path)
df


,question,answer
0,Rose went to the store on Monday and bought 4 ...,On Monday she bought 4 cakes\nOn Tuesday she b...
1,Paul needs 63 cupcakes for a birthday party ha...,The total number of chocolate and toffee cupca...
2,John fills a 6 foot by 4 foot pool that is 5 f...,The pool has a base of 6*4=<<6*4=24>>24 square...
3,John buys 2 pairs of shoes for each of his 3 c...,He bought 3*2=<<3*2=6>>6 pairs of shoes\nSo he...
4,Emma can make and upload 72 vlogs per month. B...,Emma was able to make 18 + 21 + 15 = <<18+21+1...
...,...,...
1314,A merchant wants to make a choice of purchase ...,"If he purchases jewelry, he will make a profit..."
1315,Bud makes homemade macaroni and cheese once a ...,The gruyere cheese is twice the cost of the $3...
1316,An electronics seller bought 5 phones for $700...,The seller bought the 5 phones for $700 * 5 = ...
1317,"Karan borrowed $3,650 for five months at an in...","Karan's interest is $3,650 × 0.1 = $<<3650*0.1..."


In [5]:
# Add unique key (integer index)
df['key'] = df.index

# Add dict as string
df['key_question_dict'] = df.apply(lambda row: {"key": row['key'], "question": row['question']}, axis=1)

# Add num_steps
df['num_steps'] = df['answer'].apply(lambda x: str(x).count('\n'))

# Add steps-only column
df['steps'] = df['answer'].apply(lambda x: str(x).split('###')[0].strip())

# Add final answer column
df['final_answer'] = df['answer'].apply(
    lambda x: str(x).split('###')[-1].strip() if '###' in str(x) else ''
)

df

,question,answer,key,key_question_dict,num_steps,steps,final_answer
0,Rose went to the store on Monday and bought 4 ...,On Monday she bought 4 cakes\nOn Tuesday she b...,0,"{'key': 0, 'question': 'Rose went to the store...",4,On Monday she bought 4 cakes\nOn Tuesday she b...,# 76
1,Paul needs 63 cupcakes for a birthday party ha...,The total number of chocolate and toffee cupca...,1,"{'key': 1, 'question': 'Paul needs 63 cupcakes...",2,The total number of chocolate and toffee cupca...,# 15
2,John fills a 6 foot by 4 foot pool that is 5 f...,The pool has a base of 6*4=<<6*4=24>>24 square...,2,"{'key': 2, 'question': 'John fills a 6 foot by...",3,The pool has a base of 6*4=<<6*4=24>>24 square...,# 12
3,John buys 2 pairs of shoes for each of his 3 c...,He bought 3*2=<<3*2=6>>6 pairs of shoes\nSo he...,3,"{'key': 3, 'question': 'John buys 2 pairs of s...",2,He bought 3*2=<<3*2=6>>6 pairs of shoes\nSo he...,# 360
4,Emma can make and upload 72 vlogs per month. B...,Emma was able to make 18 + 21 + 15 = <<18+21+1...,4,"{'key': 4, 'question': 'Emma can make and uplo...",2,Emma was able to make 18 + 21 + 15 = <<18+21+1...,# 18
...,...,...,...,...,...,...,...
1314,A merchant wants to make a choice of purchase ...,"If he purchases jewelry, he will make a profit...",1314,"{'key': 1314, 'question': 'A merchant wants to...",3,"If he purchases jewelry, he will make a profit...",# 125
1315,Bud makes homemade macaroni and cheese once a ...,The gruyere cheese is twice the cost of the $3...,1315,"{'key': 1315, 'question': 'Bud makes homemade ...",3,The gruyere cheese is twice the cost of the $3...,# 520
1316,An electronics seller bought 5 phones for $700...,The seller bought the 5 phones for $700 * 5 = ...,1316,"{'key': 1316, 'question': 'An electronics sell...",2,The seller bought the 5 phones for $700 * 5 = ...,# 500
1317,"Karan borrowed $3,650 for five months at an in...","Karan's interest is $3,650 × 0.1 = $<<3650*0.1...",1317,"{'key': 1317, 'question': 'Karan borrowed $3,6...",3,"Karan's interest is $3,650 × 0.1 = $<<3650*0.1...",# 803


In [6]:
counts = df['num_steps'].value_counts().sort_index()
print(counts)


num_steps
2     326
3     370
4     298
5     174
6      88
7      40
8      20
9       2
11      1
Name: count, dtype: int64


In [9]:
import numpy as np

rng = np.random.default_rng(seed=42)  # for reproducibility

def sample_range(df, low, high, n):
    mask = (df['num_steps'] >= low) & (df['num_steps'] <= high)
    candidates = df[mask]
    # If fewer than n available, sample all of them
    n_actual = min(n, len(candidates))
    return candidates.sample(n=n_actual, random_state=42)

df_1_3  = sample_range(df, 1, 3, 15)
df_4_7  = sample_range(df, 4, 7, 15)
df_8_11 = sample_range(df, 8, 11, 15)

df_sample = pd.concat([df_1_3, df_4_7, df_8_11]).sample(frac=1, random_state=42).reset_index(drop=True)

def difficulty_from_steps(n):
    if 1 <= n <= 3:
        return "easy"
    elif 4 <= n <= 7:
        return "medium"
    elif 8 <= n <= 11:
        return "hard"
    else:
        return "other"  # or np.nan

df_sample["difficulty"] = df_sample["num_steps"].apply(difficulty_from_steps)



In [10]:
df_sample

,question,answer,key,key_question_dict,num_steps,steps,final_answer,difficulty
0,"Alex, Stan, and Adelwolfe are trying to catch ...",Let x represent the number of Pokemon Adelwolf...,65,"{'key': 65, 'question': 'Alex, Stan, and Adelw...",8,Let x represent the number of Pokemon Adelwolf...,# 147,hard
1,Janet is collecting the results of Herbert Hoo...,First find the pass rate for the fourth grader...,121,"{'key': 121, 'question': 'Janet is collecting ...",6,First find the pass rate for the fourth grader...,# 70,medium
2,Eliza's rate per hour for the first 40 hours s...,Eliza is entitled to 45 -40 = <<45-40=5>>5 hou...,279,"{'key': 279, 'question': 'Eliza's rate per hou...",5,Eliza is entitled to 45 -40 = <<45-40=5>>5 hou...,# 460,medium
3,A tank has a capacity of 18000 gallons. Wanda ...,"On the first day, Wanda pumped 1/4*18000 gallo...",146,"{'key': 146, 'question': 'A tank has a capacit...",8,"On the first day, Wanda pumped 1/4*18000 gallo...",# 6000,hard
4,A nurses’ station orders bandages in bulk pack...,Let B be the number of bandages the nurses’ st...,771,"{'key': 771, 'question': 'A nurses’ station or...",8,Let B be the number of bandages the nurses’ st...,# 19,hard
5,"Bahati, Azibo, and Dinar each contributed to t...",Let B = the number of points Bahati scored\nAz...,1205,"{'key': 1205, 'question': 'Bahati, Azibo, and ...",9,Let B = the number of points Bahati scored\nAz...,# 5,hard
6,Ricardo grows tomatoes and eggplants in his ga...,Ricardo can get 22 x 5 = <<22*5=110>>110 tomat...,1061,"{'key': 1061, 'question': 'Ricardo grows tomat...",3,Ricardo can get 22 x 5 = <<22*5=110>>110 tomat...,# 142,easy
7,Cindy's math and science books weigh 2 pounds ...,The math and science book weigh 2 pounds each ...,905,"{'key': 905, 'question': 'Cindy's math and sci...",3,The math and science book weigh 2 pounds each ...,# 17,easy
8,Ralph has a record store where people can trad...,People can bring in 2 old records = get 1 new ...,564,"{'key': 564, 'question': 'Ralph has a record s...",2,People can bring in 2 old records = get 1 new ...,# 14,easy
9,Melissa is summoned to jury duty. She spends 6...,First find Melissa's net daily pay by subtract...,820,"{'key': 820, 'question': 'Melissa is summoned ...",2,First find Melissa's net daily pay by subtract...,# 2,easy


In [11]:
df_sample.drop(columns=["key","key_question_dict"], inplace=True)


In [12]:
df_sample

,question,answer,num_steps,steps,final_answer,difficulty
0,"Alex, Stan, and Adelwolfe are trying to catch ...",Let x represent the number of Pokemon Adelwolf...,8,Let x represent the number of Pokemon Adelwolf...,# 147,hard
1,Janet is collecting the results of Herbert Hoo...,First find the pass rate for the fourth grader...,6,First find the pass rate for the fourth grader...,# 70,medium
2,Eliza's rate per hour for the first 40 hours s...,Eliza is entitled to 45 -40 = <<45-40=5>>5 hou...,5,Eliza is entitled to 45 -40 = <<45-40=5>>5 hou...,# 460,medium
3,A tank has a capacity of 18000 gallons. Wanda ...,"On the first day, Wanda pumped 1/4*18000 gallo...",8,"On the first day, Wanda pumped 1/4*18000 gallo...",# 6000,hard
4,A nurses’ station orders bandages in bulk pack...,Let B be the number of bandages the nurses’ st...,8,Let B be the number of bandages the nurses’ st...,# 19,hard
5,"Bahati, Azibo, and Dinar each contributed to t...",Let B = the number of points Bahati scored\nAz...,9,Let B = the number of points Bahati scored\nAz...,# 5,hard
6,Ricardo grows tomatoes and eggplants in his ga...,Ricardo can get 22 x 5 = <<22*5=110>>110 tomat...,3,Ricardo can get 22 x 5 = <<22*5=110>>110 tomat...,# 142,easy
7,Cindy's math and science books weigh 2 pounds ...,The math and science book weigh 2 pounds each ...,3,The math and science book weigh 2 pounds each ...,# 17,easy
8,Ralph has a record store where people can trad...,People can bring in 2 old records = get 1 new ...,2,People can bring in 2 old records = get 1 new ...,# 14,easy
9,Melissa is summoned to jury duty. She spends 6...,First find Melissa's net daily pay by subtract...,2,First find Melissa's net daily pay by subtract...,# 2,easy


In [15]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/NLP Project/Data/gsm8k_test.csv")

# Add unique key (optional)
df["key"] = df.index

# Original step definition: count '\n'
df["num_steps"] = df["answer"].apply(lambda x: str(x).count("\n"))

# Steps-only (everything before '###')
df["steps_text"] = df["answer"].apply(
    lambda x: str(x).split("###")[0].strip()
)

# Final answer (everything after '###')
df["final_answer"] = df["answer"].apply(
    lambda x: str(x).split("###")[-1].strip() if "###" in str(x) else ""
)

# Turn steps_text into list of steps by splitting on '\n'
df["steps"] = df["steps_text"].apply(
    lambda x: [s.strip() for s in str(x).split("\n") if s.strip()]
)

# Recompute num_steps from this list (should match count('\n') + 1 when non-empty)
df["num_steps"] = df["steps"].apply(len)

# Difficulty buckets by num_steps
def difficulty_from_steps(n: int):
    if 1 <= n <= 3:
        return "easy"
    elif 4 <= n <= 7:
        return "medium"
    elif 8 <= n <= 11:
        return "hard"
    else:
        return "other"

df["difficulty"] = df["num_steps"].apply(difficulty_from_steps)

# Check how many we actually have per difficulty
print(df["difficulty"].value_counts().sort_index())

# Safe sampling helper
def safe_sample(sub_df, n, seed=42):
    k = min(n, len(sub_df))
    return sub_df.sample(n=k, random_state=seed)

RNG_SEED = 42
easy_df   = safe_sample(df[df["difficulty"] == "easy"],   15)
medium_df = safe_sample(df[df["difficulty"] == "medium"], 15)
hard_df   = safe_sample(df[df["difficulty"] == "hard"],   15)

df_sample = pd.concat([easy_df, medium_df, hard_df]) \
             .sample(frac=1, random_state=RNG_SEED) \
             .reset_index(drop=True)

df_sample = df_sample[["key", "question", "steps", "final_answer", "num_steps", "difficulty"]]

print(len(df_sample))
print(df_sample["difficulty"].value_counts())


difficulty
easy      697
hard       23
medium    599
Name: count, dtype: int64
45
difficulty
hard      15
medium    15
easy      15
Name: count, dtype: int64


In [16]:
df_sample

,key,question,steps,final_answer,num_steps,difficulty
0,65,"Alex, Stan, and Adelwolfe are trying to catch ...",[Let x represent the number of Pokemon Adelwol...,# 147,8,hard
1,121,Janet is collecting the results of Herbert Hoo...,[First find the pass rate for the fourth grade...,# 70,6,medium
2,279,Eliza's rate per hour for the first 40 hours s...,[Eliza is entitled to 45 -40 = <<45-40=5>>5 ho...,# 460,5,medium
3,146,A tank has a capacity of 18000 gallons. Wanda ...,"[On the first day, Wanda pumped 1/4*18000 gall...",# 6000,8,hard
4,771,A nurses’ station orders bandages in bulk pack...,[Let B be the number of bandages the nurses’ s...,# 19,8,hard
5,1205,"Bahati, Azibo, and Dinar each contributed to t...","[Let B = the number of points Bahati scored, A...",# 5,9,hard
6,603,James wants to hang crepe-paper streamers from...,[First figure out how much crepe paper James n...,# 70,3,easy
7,372,"Although Soledad works in a windowless office,...","[June has 30 days, so Soledad must hike 9,300 ...",# 60,3,easy
8,564,Ralph has a record store where people can trad...,[People can bring in 2 old records = get 1 new...,# 14,2,easy
9,281,A wooden bridge can carry no more than 5000 po...,[The boxes can weigh up to 5000 pounds - 3755 ...,# 83,2,easy


In [21]:

# save df_sample to CSV (no index column)
df_sample.to_csv('/content/drive/MyDrive/NLP Project/Evaluation/gsm8k_evaluation_set.csv', index=False)


import os

# 1) Choose a full path (EDIT THIS to where you want it saved)
file_path = "/content/drive/MyDrive/NLP Project/Evaluation/gsm8k_evaluation_set.csv"  # Mac/Linux
# or on Windows, for example:
# file_path = r"C:\Users\YourName\Downloads\gsm8k_45_sample.csv"

# 2) Save the DataFrame
df_sample.to_csv(file_path, index=False)

# 3) Print to verify where it went
print("Saved to:", file_path)
print("Current working directory:", os.getcwd())


Saved to: /content/drive/MyDrive/NLP Project/Evaluation/gsm8k_evaluation_set.csv
Current working directory: /content
